In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

2025-05-26 17:00:22.211111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748278822.258925    2080 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748278822.284521    2080 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 17:00:22.362988: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Add

def build_cnn_lstm_model():
    inputs = tf.keras.Input(shape=(18, 1))  # Shape: (timesteps, features)

    # CNN part
    x = layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # After first Conv1D and MaxPooling
    shortcut = x
    x = layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(x)
    x = Add()([shortcut, x])  # Residual connection

    # LSTM part
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, activation="tanh"))(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, activation="tanh"))(x)  # Final LSTM layer
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Dense layers
    x = layers.Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(1)(x)  # Regression output (e.g., BPM)

    model = tf.keras.Model(inputs, outputs)
    return model


In [3]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [4]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

I0000 00:00:1748278827.572615    2080 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [5]:
model=build_cnn_lstm_model()

In [6]:
model.compile(optimizer=optimizer, loss='huber', metrics=['mae'])

In [7]:
import numpy as np

In [8]:
X1=np.load("input1_reshaped.npy")
Y1=np.load("out1_reshaped.npy")

X2=np.load("input2_reshaped.npy")
Y2=np.load("out2_reshaped.npy")

In [9]:
X1

array([[[117.54315336],
        [117.54315336],
        [117.2005136 ],
        ...,
        [117.28275571],
        [117.28275571],
        [117.41480498]],

       [[117.41480498],
        [117.41480498],
        [117.29173759],
        ...,
        [118.23561728],
        [118.23561728],
        [118.31317935]],

       [[118.31317935],
        [118.37470463],
        [118.37470463],
        ...,
        [119.79420653],
        [120.12570888],
        [120.12570888]],

       ...,

       [[100.69482022],
        [100.69482022],
        [101.64573269],
        ...,
        [ 99.62628667],
        [ 99.62628667],
        [ 98.59385419]],

       [[ 98.59385419],
        [ 98.42204435],
        [ 98.42204435],
        ...,
        [ 97.53497031],
        [ 98.18329085],
        [ 98.18329085]],

       [[ 98.18329085],
        [ 97.99323596],
        [ 97.99323596],
        ...,
        [ 97.91814895],
        [ 97.94346222],
        [ 97.94346222]]])

In [10]:
X=np.vstack([X1,X2])
Y=np.vstack([Y1,Y2])

In [11]:
X

array([[[117.54315336],
        [117.54315336],
        [117.2005136 ],
        ...,
        [117.28275571],
        [117.28275571],
        [117.41480498]],

       [[117.41480498],
        [117.41480498],
        [117.29173759],
        ...,
        [118.23561728],
        [118.23561728],
        [118.31317935]],

       [[118.31317935],
        [118.37470463],
        [118.37470463],
        ...,
        [119.79420653],
        [120.12570888],
        [120.12570888]],

       ...,

       [[132.44560167],
        [132.29570582],
        [132.31129052],
        ...,
        [132.2343775 ],
        [132.2528441 ],
        [132.50644531]],

       [[132.49823978],
        [132.17023003],
        [132.54125   ],
        ...,
        [132.14235988],
        [132.44257552],
        [132.17748507]],

       [[132.350625  ],
        [132.33976563],
        [132.43261719],
        ...,
        [132.08227523],
        [132.08227523],
        [132.37498553]]])

In [12]:
print(X1.shape)
print(X.shape)

(2082, 18, 1)
(5859, 18, 1)


In [16]:
history=model.fit(X2,Y2,epochs=100,callbacks=callback,validation_split=0.1)

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - loss: 3.6984 - mae: 4.1672 - val_loss: 3.0466 - val_mae: 3.5004
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 103ms/step - loss: 2.9812 - mae: 3.4452 - val_loss: 2.6949 - val_mae: 3.1551
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 3.0353 - mae: 3.4962 - val_loss: 2.9598 - val_mae: 3.4064


In [17]:
model.save("bpm_model_d2.keras")